In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
pd.set_option('display.max_rows', None)  # 모든 행을 출력
pd.set_option('display.max_columns', None)  # 모든 열을 출력

In [3]:
raw = pd.read_csv('/kaggle/input/rba-dataset/rba-dataset.csv')

In [7]:
raw.columns

Index(['index', 'Login Timestamp', 'User ID', 'Round-Trip Time [ms]',
       'IP Address', 'Country', 'Region', 'City', 'ASN', 'User Agent String',
       'Browser Name and Version', 'OS Name and Version', 'Device Type',
       'Login Successful', 'Is Attack IP', 'Is Account Takeover'],
      dtype='object')

In [32]:
data.isnull().sum()

Login Timestamp                0
User ID                        0
Round-Trip Time [ms]    29993329
IP Address                     0
Country                        0
Region                     47409
City                        8590
Login Successful               0
Is Attack IP                   0
Is Account Takeover            0
Is Threat                      0
dtype: int64

In [33]:
len(data)

31269264

In [8]:
data = raw.drop(['index','ASN', 'User Agent String','Browser Name and Version', 'OS Name and Version', 'Device Type'], axis=1)

In [9]:
# 두 컬럼 중 하나라도 True라면 결과를 True로 설정
data['Is Threat'] = data['Is Account Takeover'] | data['Is Attack IP']

In [10]:
data.head(20)

,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,Login Successful,Is Attack IP,Is Account Takeover,Is Threat
0,2020-02-03 12:43:30.772,-4324475583306591935,NaN,10.0.65.171,NO,-,-,False,False,False,False
1,2020-02-03 12:43:43.549,-4324475583306591935,NaN,194.87.207.6,AU,-,-,False,False,False,False
2,2020-02-03 12:43:55.873,-3284137479262433373,NaN,81.167.144.58,NO,Vestland,Urangsvag,True,False,False,False
3,2020-02-03 12:43:56.180,-4324475583306591935,NaN,170.39.78.152,US,-,-,False,False,False,False
4,2020-02-03 12:43:59.396,-4618854071942621186,NaN,10.0.0.47,US,Virginia,Ashburn,False,True,False,True
5,2020-02-03 12:44:05.160,-4324475583306591935,NaN,209.236.123.126,US,-,-,False,True,False,True
6,2020-02-03 12:44:07.893,7246533443898239661,NaN,80.202.228.214,NO,-,-,True,False,False,False
7,2020-02-03 12:44:12.799,-3243978724802435038,NaN,170.39.78.177,US,-,-,False,False,False,False
8,2020-02-03 12:44:17.474,8076000552587369902,NaN,10.0.61.212,NO,-,-,False,False,False,False
9,2020-02-03 12:44:19.071,-3065936140549856249,NaN,92.221.109.162,NO,Rogaland,Sandnes,True,False,False,False


In [ ]:
data['Country'].value_counts()

In [ ]:
data.head()

In [26]:
# 나라별 공격 IP 비율과 개수 계산
grouped_data = data.groupby('Country')['Is Attack IP']
country_attack_rate = grouped_data.mean().reset_index()
country_attack_count = grouped_data.size().reset_index(name='Count')

# 결과를 병합
result = pd.merge(country_attack_rate, country_attack_count, on='Country')

# 비율을 숫자로 계산
result['Attack IP Rate'] = result['Is Attack IP'] * 100

# 내림차순으로 숫자 기준 정렬
result = result.sort_values('Attack IP Rate', ascending=False)

# 퍼센트 형식의 문자열 추가
result['Attack IP Rate'] = result['Attack IP Rate'].apply(lambda x: f"{x:.2f}%")

print(result[['Country','Count','Attack IP Rate']])

    Country     Count Attack IP Rate
55       DM         4        100.00%
225      YT        12         66.67%
174      RO    121535         49.64%
27       BO      1939         43.22%
16       BB       145         42.76%
197      TG        87         40.23%
219      VN     20245         36.11%
101      JM       438         34.70%
0        AE      2325         33.51%
228      ZW      1306         29.86%
211      US   8685928         27.82%
30       BS       351         27.07%
78       GN       401         26.93%
42       CL     18542         25.61%
144      MW      1467         25.29%
12       AW        75         24.00%
213      UZ      2081         23.59%
172      QA       617         23.50%
5        AM      2382         23.17%
91       ID    304103         23.11%
135      MM      3224         22.61%
115      LA      1546         22.19%
60       EG      2132         21.58%
96       IQ      5030         19.76%
76       GI       658         19.45%
4        AL      2892         19.26%
3

In [23]:
# 나라별 계정 탈취 비율과 개수 계산
grouped_data = data.groupby('Country')['Is Account Takeover']
country_attack_rate = grouped_data.mean().reset_index()
country_attack_count = grouped_data.size().reset_index(name='Count')

# 결과를 병합
result = pd.merge(country_attack_rate, country_attack_count, on='Country')

# 비율을 숫자로 계산
result['Attack IP Rate'] = result['Is Account Takeover'] * 100

# 내림차순으로 숫자 기준 정렬
result = result.sort_values('Attack IP Rate', ascending=False)

# 퍼센트 형식의 문자열 추가
result['Attack IP Rate'] = result['Attack IP Rate'].apply(lambda x: f"{x:.2f}%")

print(result)

    Country  Is Account Takeover     Count Attack IP Rate
173      RE         3.484321e-03       287          0.35%
174      RO         6.500185e-04    121535          0.07%
147      MZ         5.482456e-04      1824          0.05%
124      LV         4.789272e-04      6264          0.05%
42       CL         1.617948e-04     18542          0.02%
54       DK         1.517128e-04     32957          0.02%
185      SK         1.157407e-04      8640          0.01%
36       CA         5.849131e-05    153869          0.01%
99       IT         4.713900e-05    148497          0.00%
91       ID         2.959524e-05    304103          0.00%
51       CZ         1.695045e-05    117991          0.00%
29       BR         7.620009e-06   1181101          0.00%
8        AR         7.507395e-06    133202          0.00%
152      NL         6.016485e-06    166210          0.00%
153      NO         7.566016e-07  13216996          0.00%
166      PM         0.000000e+00         3          0.00%
156      NU   

In [27]:
# 나라별 공격 전체 IP 비율과 개수 계산
grouped_data = data.groupby('Country')['Is Threat']
country_attack_rate = grouped_data.mean().reset_index()
country_attack_count = grouped_data.size().reset_index(name='Count')

# 결과를 병합
result = pd.merge(country_attack_rate, country_attack_count, on='Country')

# 비율을 숫자로 계산
result['Is Threat Rate'] = result['Is Threat'] * 100

# 내림차순으로 숫자 기준 정렬
result = result.sort_values('Is Threat Rate', ascending=False)

# 퍼센트 형식의 문자열 추가
result['Is Threat'] = result['Is Threat Rate'].apply(lambda x: f"{x:.2f}%")

print(result)

    Country Is Threat     Count  Is Threat Rate
55       DM   100.00%         4      100.000000
225      YT    66.67%        12       66.666667
174      RO    49.65%    121535       49.648249
27       BO    43.22%      1939       43.218154
16       BB    42.76%       145       42.758621
197      TG    40.23%        87       40.229885
219      VN    36.11%     20245       36.107681
101      JM    34.70%       438       34.703196
0        AE    33.51%      2325       33.505376
228      ZW    29.86%      1306       29.862175
211      US    27.82%   8685928       27.824672
30       BS    27.07%       351       27.065527
78       GN    26.93%       401       26.932668
42       CL    25.63%     18542       25.628303
144      MW    25.29%      1467       25.289707
12       AW    24.00%        75       24.000000
213      UZ    23.59%      2081       23.594426
172      QA    23.50%       617       23.500810
5        AM    23.17%      2382       23.173804
91       ID    23.12%    304103       23

In [29]:
data.isnull().sum()

Login Timestamp                0
User ID                        0
Round-Trip Time [ms]    29993329
IP Address                     0
Country                        0
Region                     47409
City                        8590
Login Successful               0
Is Attack IP                   0
Is Account Takeover            0
Is Threat                      0
dtype: int64

In [36]:
Round_cleaned = data.dropna(subset=['Round-Trip Time [ms]'])

In [40]:
Round_cleaned.head()

,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,Login Successful,Is Attack IP,Is Account Takeover,Is Threat
15,2020-02-03 12:44:30.475,-8296667206273764769,1123.0,82.147.58.93,NO,Oslo County,Oslo,True,False,False,False
41,2020-02-03 12:44:57.260,-7004878831669814705,30.0,81.167.44.40,NO,Oslo County,Oslo,True,False,False,False
51,2020-02-03 12:45:11.594,5024543460394389650,476.0,84.234.159.249,NO,Oslo County,Oslo,False,False,False,False
76,2020-02-03 12:45:25.711,9006020809380285912,467.0,51.175.159.252,NO,Oslo County,Oslo,True,False,False,False
108,2020-02-03 12:45:53.556,-5076624984569379766,413.0,10.0.61.93,NO,Oslo County,Oslo,True,False,False,False


In [42]:
Round_cleaned.groupby('Is Threat')['Round-Trip Time [ms]'].mean()

Is Threat
False    662.178166
True     773.223346
Name: Round-Trip Time [ms], dtype: float64